# Box Regression - 1. Data Generation

### Table of Contents

* [Dataset](#datasetgeneration)
    * [Librairies](#librairies)
    * [Parameters](#parameters)
    * [Folder Structure](#folderstructure)
    * [Data Generation](#datageneration)
        * [Visualization](#visualization)
        * [Random Box](#randombox)
        * [Image Creation](#creation)
        * [Dataset Creation](#dataset)
    * [Main](#main)

**Work inspired by** ***Sanjeev Tripathi*** : https://github.com/sanjeev309/synthetic_bbox_regression_db_tool
## Dataset <a class="anchor" id="datasetgeneration"></a>

### Libraries  <a class="anchor" id="librairies"></a>

In [1]:
import numpy as np
from tqdm import tqdm
import random
import shutil
import cv2
import sys
import os

### Parameters  <a class="anchor" id="parameters"></a>

Some parameters to generate data :
- **SEED** : A seed to reproduce the data (dataset generated with seed = 52050)
- **PATH** : Directory of the project (storing dataset)
- **IMAGE_DIMENSION** : Dimension of the image generated (squarred images)
- **TOTAL_IMAGES** : Total number of image generated
- **TRAIN_VALID_TEST** : Tuple to indicate the repartition of data
- **CREATE_DATA** :  Boolean to indicate if the data must created
- **SAVE_IMAGE** :  Boolean to indicate if the data must be saved
- **VISUALIZATIONE** :  Boolean to indicate if the data must be shown at generation
- **AUTO_VISU** : Boolean to indicate if the visualization is in real time (if False, need to press key to generate next image)
- **DEBUG** : Boolean to indicate if additionnal print are needed (generally for debug purpose)

In [13]:
SEED = 52050

PATH = "D:/Aurelien/Documents/Aurelien/University/Master 1 (2021-2022)/MA1 - Q2/Projets/Box Regression"
os.chdir(PATH)

IMAGE_DIMENSION = 128
TOTAL_IMAGES = 10000
TRAIN_VALID_TEST = (8, 1, 1)

CREATE_DATA = True
SAVE_IMAGE = True
VISUALIZATION = True
AUTO_VISU = True
DEBUG = True

np.random.seed(SEED)

### Folder Structure  <a class="anchor" id="folderstructure"></a>

Recreates folders to store the dataset

In [4]:
def create_folder(folder):
    try:
        os.makedirs(folder)
    except:
        print('Already existing folder', file=sys.stderr)
        
if CREATE_DATA:
    # Create Folder (--force replacement)
    try:
        shutil.rmtree('db')
    except:
        print('Error deleting directory', file=sys.stderr)
    create_folder('db')
    create_folder('db/train')
    create_folder('db/valid')
    create_folder('db/test')

Error deleting directory


### Data Generation  <a class="anchor" id="datageneration"></a>

#### Visualization  <a class="anchor" id="visualization"></a>

View images as they are generated.

In [6]:
def visualize_on_spot(image, auto = AUTO_VISU, ms_timing = 10):
    # Visualization of data on the spot
    winname = "Visualization of data"
    cv2.imshow(winname, image)
    
    # If not auto, need to press key to go to next data
    if not auto:
        ms_timing = -1    
    cv2.waitKey(ms_timing)

#### Random box  <a class="anchor" id="randombox"></a>

Generate a box of random size inside the image and return the upper left corner, width and height.

In [1]:
def random_box(x_size, y_size):
    # Return a random box inside the image of size (x_size, y_size)
    w = np.random.randint(0.1*x_size, x_size + 1)
    h = np.random.randint(0.1*x_size, y_size + 1)
    x = np.random.randint(0, x_size - w + 1)
    y = np.random.randint(0, y_size - h + 1)
    
    return [x, y, w, h]

#### Image Creation <a class="anchor" id="creation"></a>

Noise function is currently not used (optionnal)

In [7]:
def add_noise(image):
    threshold = np.random.random()
    for r in range(len(image[0])):
        for c in range(len(image[1])):
            if  np.random.random() > threshold:
                image[r][c] = 255 * np.random.random()
    return image

Create and store a binary jpg image with labels (=coordinates of the box)

In [8]:
def create_data(x_size, y_size, category , idx, create=False):
    # Create black images with random white box (black = 0, white = 255, thickness=-1 for complete filling )
    x, y, w, h = random_box(x_size, y_size)
    image = np.zeros([x_size, y_size])
    cv2.rectangle(image, (x, y), (x + w, y + h), 255, -1)
    #image = add_noise(image)
    
    # Visualization & Storage of images for further use
    if create:
        if VISUALIZATION:
            visualize_on_spot(image)
        if SAVE_IMAGE:
            cv2.imwrite("db/" + category + "/" + category + "_" + str(idx) + ".jpg", image)
        dbg_len = len(str(IMAGE_DIMENSION))
        dbg_nbr = len(str(TOTAL_IMAGES))
        with open("db/" + category + "/.debug_" + category + "_data.txt", 'a') as f:
            print(f"Image {idx:>{dbg_nbr}}: ({x:>{dbg_len}}, {y:>{dbg_len}})" 
                  f" -> ({x + w:>{dbg_len}}, {y + h:>{dbg_len}})", file=f)
    
    return image, x, y, w, h

#### Dataset Creation <a class="anchor" id="dataset"></a>

Create and store a dataset with labels (= coordinates of the upper left and lower right corner)

In [9]:
def generate_dataset(category, nbr_data):
    
    if DEBUG:
        dbg_len = len(str(IMAGE_DIMENSION))
        dbg_nbr = len(str(nbr_data))
    
    # Initialization of data (grayscale images) & labels (x,y,w,h) arrays
    data = np.empty([0, IMAGE_DIMENSION, IMAGE_DIMENSION, 1], dtype=np.int64)
    target = np.empty([0, 4], dtype=np.int64)

    # Generation of data & labels
    with open("db/" + category + "/.debug_" + category + "_data.txt", 'a') as f:
            print(f"# Data details of type : {category}"
                  f"# Bounding Box : (upper left corner) -> (lower right corner) : (x1, y2) -> (x2, y2)", file=f)
    for i in tqdm(range(nbr_data), desc="Create " + category + " data..."):
        im, x, y, w, h = create_data(IMAGE_DIMENSION, IMAGE_DIMENSION, category, i, CREATE_DATA)
        data = np.vstack((data, [np.expand_dims(im, axis=2)]))
        target = np.vstack((target, [x, y, x + w, y + h]))
    
    # Normalization
    target = target / IMAGE_DIMENSION
    data = data / 255
    
    # Save & close visualization
    np.save("db/" + category + "/." + category + "_data", data)
    np.save("db/" + category + "/." + category + "_target", target)
    cv2.destroyAllWindows()

### Main <a class="anchor" id="main"></a>

Main loop to generate the whole dataset.

In [10]:
if CREATE_DATA:
    train_ratio = TRAIN_VALID_TEST[0] / sum(TRAIN_VALID_TEST)
    valid_ratio = TRAIN_VALID_TEST[1] / sum(TRAIN_VALID_TEST)
    test_ratio = TRAIN_VALID_TEST[2] / sum(TRAIN_VALID_TEST)
    
    generate_dataset("train", int(TOTAL_IMAGES * train_ratio))
    generate_dataset("valid", int(TOTAL_IMAGES * valid_ratio))
    generate_dataset("test", int(TOTAL_IMAGES * test_ratio))

Create test data...: 100%|█████████████████████████████████████████████████████████| 1000/1000 [01:12<00:00, 13.89it/s]
